# Spectrum

In [1]:
import pandas as pd
import astropy.units as u
import numpy as np
import re
import matplotlib.pyplot as plt
from astroquery.simbad import Simbad
from astropy.coordinates import SkyCoord
from pandas.plotting import scatter_matrix

In [2]:
catalog = pd.read_csv("markeddata/gevtevsimbad_TeVspectrum.txt", sep='\t', index_col=0)

In [3]:
catalog.head()

,level_0,gev_0FGL_Name,gev_1FGL_Name,gev_1FHL_Name,gev_2FGL_Name,gev_ASSOC1,gev_ASSOC2,gev_ASSOC_GAM1,gev_ASSOC_GAM2,gev_ASSOC_GAM3,...,tev_tevcat_name,tev_tgevcat_id,tev_tgevcat_name,tev_where,marked,tev_0.3TeV,tev_1TeV,tev_3TeV,tev_10TeV,tev_30TeV
0,83,b' ',b' ',b' ',b' ',b' ',b' ',b' ',b' ',b' ',...,b'TeV J1745-290',106,b'TeV J1745-2900',b'gal',0,3.341668e-11,2.440195e-12,1.828992e-13,5.610705e-15,6.128219e-17
1,92,b' ',b' ',b' ',b' ',b'HESS J1841-055 ',b' ',b' ',b' ',b' ',...,b'TeV J1840-055',125,b'TeV J1840-0533',b'gal',0,6.310590e-11,1.207092e-11,8.365091e-13,4.356098e-14,2.865129e-15
2,83,b' ',b' ',b' ',b' ',b' ',b' ',b' ',b' ',b' ',...,b'TeV J1745-290',106,b'TeV J1745-2900',b'gal',0,3.341668e-11,2.440195e-12,1.828992e-13,5.610705e-15,6.128219e-17
3,86,b' ',b'1FGL J1746.4-2849c',b'1FHL J1746.3-2851 ',b'2FGL J1746.6-2851c',b'PWN G0.13-0.11 ',b' ',b' ',b'3EG J1746-2851',b' ',...,b'TeV J1746-289',-9223372036854775808,b'',b'gal',0,1.647138e-11,8.320851e-14,2.778103e-14,1.856180e-15,3.184437e-16
4,83,b' ',b' ',b' ',b' ',b' ',b' ',b' ',b' ',b' ',...,b'TeV J1745-290',106,b'TeV J1745-2900',b'gal',0,3.341668e-11,2.440195e-12,1.828992e-13,5.610705e-15,6.128219e-17


Columns contained spectrum in gev:

In [4]:
gev_spectrum_columns = [
    'gev_nuFnu10000_100000',
    'gev_nuFnu1000_3000',
    'gev_nuFnu100_300',
    'gev_nuFnu3000_10000',
    'gev_nuFnu300_1000',
    'gev_nuFnu30_100',
]

Columns contained spectrum in simbad:

In [5]:
s_spectrum_columns = [
    's_FLUX_U', 
    's_FLUX_B', 
    's_FLUX_V', 
    's_FLUX_R', 
    's_FLUX_I',
    's_FLUX_G', 
    's_FLUX_J', 
    's_FLUX_H',
    's_FLUX_K', 
    's_FLUX_u',
    's_FLUX_g', 
    's_FLUX_r',
    's_FLUX_i',    
    's_FLUX_z',
    ]

Columns contained spectrum in tev:

In [6]:
tev_spectrum_columns =  [
    'tev_0.3TeV', 
    'tev_1TeV', 
    'tev_3TeV',  
    'tev_10TeV',  
    'tev_30TeV' 
]

In [7]:
s_other_columns = ['s_MAIN_ID',
                 's_class', 
                 's_nan_spectra'  
                 ]

In [8]:
gev_other_columns = ['gev_1FGL_Name',
                     'gev_CLASS1',
                    ]

In [9]:
tev_other_columns = ['tev_fermi_names', 
                     'tev_classes',
                    ]

In [10]:
spectrum_columns = gev_spectrum_columns + s_spectrum_columns + tev_spectrum_columns
other_columns = s_other_columns + gev_other_columns + tev_other_columns + ['marked']

In [11]:
spectrums = catalog[spectrum_columns + other_columns].copy()

The next cell computes simbad spectrum using this formula:
$$ F_{simbad} = 10^{-flux_{simbad}}$$

In [12]:
spectrums[s_spectrum_columns] = 10 ** (-spectrums[s_spectrum_columns])

The next cell computes TeV spectrum using:
$$F_{TeV} = 1.6*E^{2}*Flux$$

In [13]:
for i in tev_spectrum_columns:
    if len(re.findall("\d+", i)) == 2:
        E = 0.3
    else:
        E = int(re.findall("\d+", i)[0])
    spectrums[i] = 1.6 * E ** 2 * spectrums[i]

In [14]:
spectrums = spectrums.fillna(0)

In [15]:
display(spectrums)

,gev_nuFnu10000_100000,gev_nuFnu1000_3000,gev_nuFnu100_300,gev_nuFnu3000_10000,gev_nuFnu300_1000,gev_nuFnu30_100,s_FLUX_U,s_FLUX_B,s_FLUX_V,s_FLUX_R,...,tev_10TeV,tev_30TeV,s_MAIN_ID,s_class,s_nan_spectra,gev_1FGL_Name,gev_CLASS1,tev_fermi_names,tev_classes,marked
0,2.938044e-12,7.446349e-11,4.520060e-12,3.103834e-11,7.171538e-11,0.0,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,...,8.977128e-13,8.824635e-14,b'SSTGC 519103',Be,False,b' ',b' ',b'',b'unid',0
1,2.098420e-11,2.096862e-11,1.266728e-12,2.274123e-11,7.566604e-12,0.0,0.000000e+00,6.918312e-12,3.090294e-11,0.000000e+00,...,6.969757e-12,4.125786e-12,b'LS IV -05 11',B,False,b' ',b'PWN ',b'',b'unid',0
2,2.938044e-12,7.446349e-11,4.520060e-12,3.103834e-11,7.171538e-11,0.0,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,...,8.977128e-13,8.824635e-14,b'GCIRS 1W',B,False,b' ',b' ',b'',b'unid',0
3,4.706814e-12,5.332763e-11,5.145968e-13,2.923599e-11,3.387356e-11,0.0,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,...,2.969888e-13,4.585590e-13,b'[MCD2010] 18',O,False,b'1FGL J1746.4-2849c',b'pwn ',b'',b'unid',0
4,2.938044e-12,7.446349e-11,4.520060e-12,3.103834e-11,7.171538e-11,0.0,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,...,8.977128e-13,8.824635e-14,b'GCIRS 8',O,False,b' ',b' ',b'',b'unid',0
5,4.706814e-12,5.332763e-11,5.145968e-13,2.923599e-11,3.387356e-11,0.0,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,...,0.000000e+00,0.000000e+00,b'[MCD2010] 13',O,False,b'1FGL J1746.4-2849c',b'pwn ',b'',b'unid',0
6,4.706814e-12,5.332763e-11,5.145968e-13,2.923599e-11,3.387356e-11,0.0,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,...,2.969888e-13,4.585590e-13,b'[MCD2010] 13',O,False,b'1FGL J1746.4-2849c',b'pwn ',b'',b'unid',0
7,2.845686e-12,4.600289e-11,1.418847e-11,3.224835e-11,3.170300e-11,0.0,7.585778e-14,8.317639e-14,1.122019e-12,0.000000e+00,...,0.000000e+00,0.000000e+00,b'[MT91] 213',B0Ve,False,b'1FGL J2032.2+4127 ',b'PSR ',b'',b'unid',1
8,2.845686e-12,4.600289e-11,1.418847e-11,3.224835e-11,3.170300e-11,0.0,4.073800e-13,9.772367e-13,5.011870e-12,8.912505e-12,...,0.000000e+00,0.000000e+00,b'Schulte 21',B0Ve,False,b'1FGL J2032.2+4127 ',b'PSR ',b'',b'unid',0
9,5.929003e-12,8.840736e-11,1.353836e-10,4.055951e-11,1.317918e-10,0.0,5.370312e-12,2.454711e-12,1.778279e-11,6.456549e-11,...,2.910345e-13,4.065659e-13,b'LS I +61 303',B0Ve,False,b'1FGL J0240.5+6113 ',b'HMB ',b'',b'bin',1


Normalization:

In [16]:
spectrums[spectrum_columns] = spectrums[spectrum_columns].div(spectrums.sum(axis=1), axis=0)

The cell displays all information we have about spectrums now.

In [17]:
display(spectrums)

,gev_nuFnu10000_100000,gev_nuFnu1000_3000,gev_nuFnu100_300,gev_nuFnu3000_10000,gev_nuFnu300_1000,gev_nuFnu30_100,s_FLUX_U,s_FLUX_B,s_FLUX_V,s_FLUX_R,...,tev_10TeV,tev_30TeV,s_MAIN_ID,s_class,s_nan_spectra,gev_1FGL_Name,gev_CLASS1,tev_fermi_names,tev_classes,marked
0,1.353665e-02,3.430807e-01,2.082558e-02,1.430050e-01,3.304191e-01,0.0,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,...,4.136093e-03,4.065834e-04,b'SSTGC 519103',Be,False,b' ',b' ',b'',b'unid',0
1,1.333478e-02,1.332488e-02,8.049650e-04,1.445132e-02,4.808333e-03,0.0,0.000000e+00,4.396364e-03,1.963782e-02,0.000000e+00,...,4.429056e-03,2.621804e-03,b'LS IV -05 11',B,False,b' ',b'PWN ',b'',b'unid',0
2,1.681789e-03,4.262423e-02,2.587363e-03,1.776690e-02,4.105117e-02,0.0,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,...,5.138669e-04,5.051379e-05,b'GCIRS 1W',B,False,b' ',b' ',b'',b'unid',0
3,1.139862e-02,1.291450e-01,1.246213e-03,7.080159e-02,8.203253e-02,0.0,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,...,7.192259e-04,1.110505e-03,b'[MCD2010] 18',O,False,b'1FGL J1746.4-2849c',b'pwn ',b'',b'unid',0
4,1.280898e-02,3.246382e-01,1.970608e-02,1.353177e-01,3.126572e-01,0.0,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,...,3.913755e-03,3.847272e-04,b'GCIRS 8',O,False,b' ',b' ',b'',b'unid',0
5,5.245202e-03,5.942750e-02,5.734588e-04,3.258014e-02,3.774818e-02,0.0,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,...,0.000000e+00,0.000000e+00,b'[MCD2010] 13',O,False,b'1FGL J1746.4-2849c',b'pwn ',b'',b'unid',0
6,5.223892e-03,5.918606e-02,5.711290e-04,3.244777e-02,3.759482e-02,0.0,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,...,3.296152e-04,5.089350e-04,b'[MCD2010] 13',O,False,b'1FGL J1746.4-2849c',b'pwn ',b'',b'unid',0
7,2.845686e-12,4.600289e-11,1.418847e-11,3.224835e-11,3.170300e-11,0.0,7.585778e-14,8.317639e-14,1.122019e-12,0.000000e+00,...,0.000000e+00,0.000000e+00,b'[MT91] 213',B0Ve,False,b'1FGL J2032.2+4127 ',b'PSR ',b'',b'unid',1
8,7.519406e-04,1.215575e-02,3.749144e-03,8.521264e-03,8.377162e-03,0.0,1.076456e-04,2.582238e-04,1.324330e-03,2.355030e-03,...,0.000000e+00,0.000000e+00,b'Schulte 21',B0Ve,False,b'1FGL J2032.2+4127 ',b'PSR ',b'',b'unid',0
9,5.929002e-12,8.840736e-11,1.353836e-10,4.055951e-11,1.317918e-10,0.0,5.370312e-12,2.454711e-12,1.778279e-11,6.456548e-11,...,2.910344e-13,4.065659e-13,b'LS I +61 303',B0Ve,False,b'1FGL J0240.5+6113 ',b'HMB ',b'',b'bin',1


In [18]:
spectrums.to_csv("markeddata/normalized_spectra.txt", index=False)

In [19]:
def calculate_spectra(catalog_name, s_spectrum_columns, gev_spectrum_columns, tev_spectrum_columns,
                      s_other_columns, gev_other_columns, tev_other_columns):
    other_columns = s_other_columns + gev_other_columns + tev_other_columns + ['marked']
    catalog = pd.read_csv("markeddata/" + catalog_name + ".txt", sep='\t', index_col=0)
    spectrum_columns = gev_spectrum_columns + s_spectrum_columns + tev_spectrum_columns
    spectrums = catalog[spectrum_columns + other_columns].copy()
    if len(s_spectrum_columns) > 0:
        spectrums[s_spectrum_columns] = 10 ** (-spectrums[s_spectrum_columns])
    for i in tev_spectrum_columns:
        if len(re.findall("\d+", i)) == 2:
            E = 0.3
        else:
            E = int(re.findall("\d+", i)[0])
        spectrums[i] = 1.6 * E ** 2 * spectrums[i]
    spectrums = spectrums.fillna(0)
    spectrums[spectrum_columns] = spectrums[spectrum_columns].div(spectrums.sum(axis=1), axis=0)
    spectrums = spectrums.fillna(0)
    display(spectrums)
    spectrums.to_csv("markeddata/" + catalog_name + "_normalized.txt", index=False)

In [20]:
calculate_spectra(catalog_name="gevtev_TeVspectrum", 
                  s_spectrum_columns=[], 
                  gev_spectrum_columns=gev_spectrum_columns, 
                  tev_spectrum_columns=tev_spectrum_columns, 
                  s_other_columns=[],
                  gev_other_columns=gev_other_columns,
                  tev_other_columns=tev_other_columns)
calculate_spectra(catalog_name="tevsimbad_TeVspectrum", 
                  s_spectrum_columns =s_spectrum_columns, 
                  gev_spectrum_columns=[], 
                  tev_spectrum_columns=tev_spectrum_columns, 
                  s_other_columns=s_other_columns,
                  gev_other_columns=[],
                  tev_other_columns=tev_other_columns)
calculate_spectra(catalog_name="gevsimbad", 
                  s_spectrum_columns=s_spectrum_columns, 
                  gev_spectrum_columns=gev_spectrum_columns, 
                  tev_spectrum_columns=[], 
                  s_other_columns=s_other_columns,
                  gev_other_columns=gev_other_columns,
                  tev_other_columns=[])

,gev_nuFnu10000_100000,gev_nuFnu1000_3000,gev_nuFnu100_300,gev_nuFnu3000_10000,gev_nuFnu300_1000,gev_nuFnu30_100,tev_0.3TeV,tev_1TeV,tev_3TeV,tev_10TeV,tev_30TeV,gev_1FGL_Name,gev_CLASS1,tev_fermi_names,tev_classes,marked
0,5.515514e-02,2.014771e-01,4.369501e-06,2.350665e-01,2.824920e-02,0.0,3.916187e-01,7.284051e-02,1.319284e-02,2.028336e-03,3.673711e-04,b' ',b'bll ',b'',b'hbl',0
1,6.911902e-02,7.317161e-02,3.406560e-03,3.832461e-02,1.456153e-01,0.0,1.340714e-01,2.042935e-02,6.451667e-03,5.937940e-02,4.500310e-01,b' ',b'snr ',b'',b'snr',0
2,3.581899e-01,1.781788e-01,9.673682e-02,2.689078e-01,9.798673e-02,0.0,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,b'1FGL J0033.5-1921 ',b'bll ',b'',b'hbl',0
3,2.690050e-01,1.574623e-01,1.397244e-01,2.205611e-01,2.132472e-01,0.0,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,b'1FGL J0035.9+5951 ',b'bll ',b'',b'hbl',0
4,4.277253e-01,1.472365e-01,1.041475e-01,2.310657e-01,8.982494e-02,0.0,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,b'1FGL J0136.5+3905 ',b'bll ',b'',b'hbl',0
5,1.848032e-01,1.392944e-01,9.296665e-02,7.216214e-02,6.466378e-02,0.0,2.709071e-01,7.249194e-02,4.900784e-02,3.200621e-02,2.169677e-02,b' ',b'bll ',b'',b'hbl',0
6,6.631103e-03,2.356757e-01,3.527876e-01,8.705421e-02,3.178513e-01,0.0,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,b'1FGL J0205.6+6449 ',b'PSR ',b'',b'pwn',0
7,8.001306e-02,1.673595e-01,3.639620e-01,1.393246e-01,2.493409e-01,0.0,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,b'1FGL J0221.0+3555 ',b'FSRQ ',b'',b'blazar',0
8,1.932148e-01,2.107472e-01,1.453232e-01,2.300573e-01,1.928651e-01,0.0,2.103588e-02,5.001267e-03,1.348328e-03,3.205642e-04,8.642325e-05,b'1FGL J0222.6+4302 ',b'BLL ',b'',b'ibl',0
9,1.987382e-01,2.167718e-01,1.494776e-01,2.366340e-01,1.983785e-01,0.0,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,b'1FGL J0222.6+4302 ',b'BLL ',b'',b'unid',0


,s_FLUX_U,s_FLUX_B,s_FLUX_V,s_FLUX_R,s_FLUX_I,s_FLUX_G,s_FLUX_J,s_FLUX_H,s_FLUX_K,s_FLUX_u,...,tev_1TeV,tev_3TeV,tev_10TeV,tev_30TeV,s_MAIN_ID,s_class,s_nan_spectra,tev_fermi_names,tev_classes,marked
0,0.000000e+00,0.000000e+00,0.000000e+00,6.038388e-05,0.000000,1.434899e-04,1.701848e-03,3.242803e-03,7.778944e-03,0.0,...,2.726754e-01,5.690167e-02,3.731492e-02,1.353685e-01,b'UCAC4 105-013663',Be,False,b'',b'other',0
1,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000,0.000000e+00,3.376672e-04,1.956543e-02,8.699397e-01,0.0,...,2.901023e-02,2.593981e-02,8.455093e-03,9.924860e-04,b'MSX6C G331.5487-00.5364',Be,False,b'',b'unid',0
2,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000,0.000000e+00,8.607538e-06,2.454024e-03,6.164239e-01,0.0,...,1.206213e-01,8.136797e-02,2.773427e-02,2.726315e-03,b'SSTGC 519103',Be,False,b'',b'unid',0
3,0.000000e+00,2.770727e-02,4.598265e-02,0.000000e+00,0.000000,0.000000e+00,1.326154e-01,1.708416e-01,2.053968e-01,0.0,...,2.301939e-01,1.369295e-01,4.578426e-02,1.109544e-03,b'RMC 141',B,False,b'',b'pwn',0
4,0.000000e+00,5.309586e-04,6.181027e-04,3.428160e-04,0.000474,6.175343e-04,3.846461e-04,2.600526e-04,2.443773e-04,0.0,...,2.752872e-01,5.744670e-02,3.767234e-02,1.366652e-01,b'[AAA97b] J053648.7-691700',B,False,b'',b'other',0
5,0.000000e+00,1.827689e-05,2.197365e-05,2.248550e-05,0.000042,3.257645e-05,1.555617e-05,1.628933e-05,1.451784e-05,0.0,...,5.513188e-01,3.279489e-01,1.096542e-01,2.657379e-03,b'OGLE LMC-ECL-20994',B,False,b'',b'pwn',0
6,9.568497e-03,1.909167e-03,1.624966e-03,0.000000e+00,0.000000,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.0,...,2.726270e-01,5.689157e-02,3.730830e-02,1.353445e-01,b'[ST92] 2-62B',B,False,b'',b'other',0
7,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000,0.000000e+00,4.518918e-03,3.311572e-02,8.571056e-02,0.0,...,2.928115e-01,1.423089e-01,4.350691e-02,1.475448e-02,b'[KKM2013] 7',B,False,b'',"b'pwn,snr'",0
8,4.386694e-03,1.871274e-03,2.643243e-02,0.000000e+00,0.000000,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.0,...,2.672159e-01,5.576239e-02,3.656781e-02,1.326582e-01,b'[ST92] 2-62',B,False,b'',b'other',0
9,2.967705e-03,9.581216e-04,3.796830e-03,9.895109e-03,0.452295,4.113587e-03,8.154943e-02,1.575510e-01,2.502773e-01,0.0,...,2.017999e-02,1.200395e-02,4.013686e-03,9.726836e-05,b'HTR 13',B,False,b'',b'pwn',0


,gev_nuFnu10000_100000,gev_nuFnu1000_3000,gev_nuFnu100_300,gev_nuFnu3000_10000,gev_nuFnu300_1000,gev_nuFnu30_100,s_FLUX_U,s_FLUX_B,s_FLUX_V,s_FLUX_R,...,s_FLUX_g,s_FLUX_r,s_FLUX_i,s_FLUX_z,s_MAIN_ID,s_class,s_nan_spectra,gev_1FGL_Name,gev_CLASS1,marked
0,1.257344e-12,2.414663e-11,8.859199e-12,1.480012e-11,2.684172e-11,0.0,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,...,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,b'[KRL2007b] 194',Be,True,b'1FGL J1636.4-4737c',b'snr ',0
1,4.367979e-06,8.042724e-05,1.744995e-05,3.972280e-05,7.974043e-05,0.0,8.700521e-05,3.626980e-05,8.119804e-05,1.022224e-04,...,4.360598e-05,1.228983e-04,1.411059e-04,0.000000e+00,b'HD 328990',Be,False,b'1FGL J1651.5-4602c',b' ',0
2,1.435131e-02,8.726880e-02,9.648044e-02,5.887288e-02,8.264021e-02,0.0,4.192119e-05,2.251299e-05,1.788273e-04,8.173980e-04,...,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,b'UCAC2 5051621',Be,False,b' ',b'spp ',0
3,5.414631e-02,1.237049e-01,4.429596e-01,1.213109e-01,1.887937e-01,0.0,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,...,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,b'SSTGC 519103',Be,False,b'1FGL J1745.6-2900c',b'spp ',0
4,4.642829e-02,2.322116e-01,2.710111e-01,1.452609e-01,3.043594e-01,0.0,0.000000e+00,2.831540e-05,2.624356e-05,0.000000e+00,...,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,b'LIN 341',Be,False,b'1FGL J0101.3-7257 ',b'GAL ',0
5,1.523725e-03,6.399917e-03,1.623092e-02,4.016086e-03,8.842059e-03,0.0,1.508756e-01,3.456357e-02,5.231411e-02,0.000000e+00,...,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,b'LS VI -04 15',B,False,b' ',b' ',0
6,1.573251e-04,2.620224e-03,8.954646e-03,1.090295e-03,3.881067e-03,0.0,0.000000e+00,6.987534e-03,1.528706e-02,0.000000e+00,...,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,b'LS I +56 58',B,False,b' ',b' ',0
7,1.435295e-05,1.313128e-01,4.130346e-01,7.424117e-02,3.705391e-01,0.0,0.000000e+00,2.135673e-04,4.462050e-04,3.158892e-04,...,4.915121e-04,4.748264e-04,4.173824e-04,3.503764e-04,b'PG 1101+529',B,False,b' ',b' ',0
8,1.144422e-05,2.401992e-01,3.359969e-01,8.313828e-02,3.247328e-01,0.0,0.000000e+00,0.000000e+00,1.592146e-02,0.000000e+00,...,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,b'GALEX J175340.5-500741',B,False,b'1FGL J1754.0-5002 ',b'bcu ',0
9,1.941257e-02,1.180458e-01,1.305061e-01,7.963548e-02,1.117848e-01,0.0,2.011987e-03,2.058849e-03,7.649360e-03,1.457555e-02,...,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,b'HD 306182',B,False,b' ',b'spp ',0
